In [1]:
from pyspark import SparkContext

In [2]:
sc=SparkContext(master="local",appName="transformacionesAcciones")

In [3]:
rdd1=sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [4]:
 rdd1.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=transformacionesAcciones>

In [6]:
!dir "D:\Proyectos Personales\Curso Spark\files"

 El volumen de la unidad D es Datos
 El número de serie del volumen es: 62E4-5D18

 Directorio de D:\Proyectos Personales\Curso Spark\files

04-02-2021  15:37    <DIR>          .
04-02-2021  15:37    <DIR>          ..
04-02-2021  15:37             1.013 deporte.csv
04-02-2021  15:37         2.832.323 deportista.csv
04-02-2021  15:37         2.874.875 deportista2.csv
04-02-2021  15:37         2.844.569 deportistaError.csv
04-02-2021  15:37            34.028 evento.csv
04-02-2021  15:37             2.030 juegos.csv
04-02-2021  15:37           255.853 modelo_relacional.jpg
04-02-2021  15:37            24.791 paises.csv
04-02-2021  15:37         6.443.913 resultados.csv
               9 archivos     15.313.395 bytes
               2 dirs  524.082.982.912 bytes libres


In [7]:
path="D:\\Proyectos Personales\\Curso Spark\\files"
##Cargar Archivo
equiposOlimpicosRDD=sc.textFile(path+"\paises.csv") \
                      .map(lambda line : line.split(","))

In [8]:
equiposOlimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [9]:
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [10]:
equiposOlimpicosRDD.map(lambda x : (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [11]:
equiposOlimpicosRDD.map(lambda x : (x[2],x[1])).groupByKey().mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [12]:
equiposArgentinos=equiposOlimpicosRDD.filter(lambda x : "ARG" in x[2])
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [13]:
equiposOlimpicosRDD.count()

1185

In [14]:
equiposOlimpicosRDD.countApprox(20)

1185

In [35]:
##Cargar Archivo
DeportistaOlimpicoRDD=sc.textFile(path+"\deportista.csv") \
                      .map(lambda line : line.split(","))

In [36]:
##Cargar Archivo
DeportistaOlimpicoRDD2=sc.textFile(path+"\deportista2.csv") \
                      .map(lambda line : line.split(","))

In [37]:
DeportistaOlimpicoRDD=DeportistaOlimpicoRDD\
                      .union(DeportistaOlimpicoRDD2)

In [38]:
DeportistaOlimpicoRDD.count()

135572

In [39]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [40]:
DeportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [91]:
''' Buscar Ultima Columna para poder hacer el Match  '''
DeportistaOlimpicoRDD.map(lambda line: [line[-1],line[:-1]])\
                     .join(equiposOlimpicosRDD.map(lambda e : [e[0],e[2]]))\
                      .take(2)
##.takeSample(False,10,25)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN'))]

In [86]:
##Cargar Archivo
ResultadoRDD=sc.textFile(path+"\\resultados.csv")\
                      .map(lambda line : line.split(","))

In [87]:
ResultadoGanadorRDD=ResultadoRDD.filter(lambda l : 'NA' not in l[1])

In [88]:
ResultadoGanadorRDD.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [94]:
DeportistaOlimpicoRDD.map(    lambda d : [d[-1], d[:-1]])\
.join(equiposOlimpicosRDD.map(lambda e : [e[0],  e[2]]))\
.join(ResultadoGanadorRDD.map(lambda r : [r[-1], r[1]]))\
.takeSample(False,10,10)

[('45',
  ((['73728', 'Laura Maiztegui', '2', '21', '163', '64'], 'ARG'), 'Silver')),
 ('470',
  ((['85652', 'Ng Wing Nga', '2', '21', '165', '53'], 'HKG'), 'Bronze')),
 ('55',
  ((['87383', 'Hajimurad Nurmagomedov', '1', '25', '173', '84'], 'ARM'),
   'Bronze')),
 ('71',
  ((['85485', 'Alexander Neumller', '1', '22', '190', '130'], 'AUT'),
   'Silver')),
 ('273',
  ((['54447', 'Sren Marinus Jensen', '1', '26', '0', '0'], 'DEN'), 'Bronze')),
 ('45',
  ((['65650', 'Ral Lujn Labbate', '1', '24', '180', '72'], 'ARG'), 'Silver')),
 ('45',
  ((['37840', 'Jorge Alberto Gaday', '1', '20', '188', '74'], 'ARG'),
   'Silver')),
 ('302',
  ((['38369', 'Reiner Ganschow', '1', '27', '184', '85'], 'GDR'), 'Silver')),
 ('45',
  ((['69761', 'Ulf Lienhard', '1', '25', '185', '73'], 'ARG'), 'Silver')),
 ('71', ((['115657', 'Gnther Stranner', '1', '20', '0', '0'], 'AUT'), 'Gold'))]